# [参考] pysofia についての調査

ランク学習が利用できるという「pysofia」というPythonベースの機械学習ソフトウェアについて調査しました。

こちらを参考にいたしました。

https://pypi.python.org/pypi/pysofia/

https://github.com/rth/pysofia

ただし、<a href="04-sofia-ml-evaluation.ipynb"><b>sofia-ml</b></a>（C言語実装版）と同様、あまりメンテナンスされていない様子で、ドキュメントも整備されておりません。

また、<a href="03-pyltr-evaluation.ipynb"><b>Python learning to rank (LTR) toolkit</b></a> が比較的良好な結果を示すのに対し、こちらは pyltr の予測結果と比較してもかけ離れているなど、sofia-ml（C言語実装版）同様、ランク学習エンジンとしては使い物にならないという印象です。

## (1) 概要

<a href="04-sofia-ml-evaluation.ipynb"><b>sofia-ml</b></a> のC言語実装版とのことで、Python 言語により実装されております。

- 実装されているのは「RankSVM」と呼ばれるランク学習アルゴリズム

- 「RankSVM」は、手法としては pairwise に分類されるようです。

　　こちら (https://en.wikipedia.org/wiki/Learning_to_rank#Pointwise_approach) の「List of methods」をご参照
  
  
- 提供されている機能

　　svm_train／svm_predict という関数で、学習／予測ができるようです。
  
　　ただしドキュメンテーションとしては以下の記述のみです。

> Trains a model using stochastic gradient descent. 
 

- 入力データ（教師データ）形式は scikit-learn と同じものが使用できるようです。

## (2) 環境準備

### (2-1) GitHub から直接インストール

すでにインストール済みの pysofia がある場合、自動的にアンインストールされるようです。

```
MacBookPro-makmorit-jp:~ makmorit$ pip3 install -U git+https://github.com/rth/pysofia.git
Collecting git+https://github.com/rth/pysofia.git
  Cloning https://github.com/rth/pysofia.git to /private/var/folders/5l/6wfffl0n1_12bxr3sysq1wgw0000gn/T/pip-mfsh5_nz-build
Requirement already up-to-date: six in /Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages (from pysofia==0.9)
Installing collected packages: pysofia
  Found existing installation: pysofia 0.8
    Uninstalling pysofia-0.8:
      Successfully uninstalled pysofia-0.8
  Running setup.py install for pysofia ... done
Successfully installed pysofia-0.9
MacBookPro-makmorit-jp:~ makmorit$ 
```

### (2-2) テストデータの準備

<a href="03-pyltr-evaluation.ipynb"><b>pyltr</b></a> の調査で使用した、Microsoft のランク学習用ベンチマークデータ（MSLR）から一部抜粋して作成したテストデータを、そのまま無変更で使用します。

MSLRのデータセットは非常に大きいので、教師データ（train.txt）と正解ラベル付きのテストデータ（vali.txt）を、それぞれ10,000件／2,500件ずつ切り出しています。

テストデータは、教師データ（train.txt）の先頭２０件を拾い出し、一番最初の列にユニークなラベルを付しています。

```
MacBookPro-makmorit-jp:pyltr makmorit$ pwd
/Users/makmorit/GitHub/pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ ls -al data
total 28536
drwxr-xr-x   7 makmorit  staff       238 May  6 12:54 .
drwxr-xr-x  15 makmorit  staff       510 May  3 15:58 ..
-rw-r--r--@  1 makmorit  staff      6148 May  6 12:54 .DS_Store
drwxr-xr-x   5 makmorit  staff       170 May  6 11:47 backup
-rwxr-----@  1 makmorit  staff     23525 May  6 12:14 test.txt
-rwxr-----@  1 makmorit  staff  11621820 May  6 11:53 train.txt
-rwxr-----@  1 makmorit  staff   2952380 May  6 11:55 vali.txt
MacBookPro-makmorit-jp:pyltr makmorit$ head -20 data/train.txt
3 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:406 12:0 13:5 14:5 15:416 （以下略）
1 qid:1 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:146 12:0 13:3 14:7 15:156 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:287 12:1 13:4 14:7 15:299 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:2009 12:2 13:4 14:7 15:2022 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:935 12:3 13:4 14:7 15:949 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1363 12:4 13:4 14:7 15:1378 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:489 12:0 13:4 14:10 15:503 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1295 12:2 13:4 14:7 15:1308 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:510 12:0 13:4 14:5 15:519 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:164 12:1 13:10 14:10 15:185 （以下略）
1 qid:1 1:3 2:1 3:3 4:0 5:3 6:1 7:0.333333 8:1 9:0 10:1 11:666 12:3 13:4 14:7 15:680 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1475 12:1 13:4 14:7 15:1487 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1601 12:0 13:1593 14:5 15:3199 （以下略）
1 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1268 12:0 13:4 14:7 15:1279 （以下略）
1 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:12 15:762 （以下略）
1 qid:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:21 12:3 13:3 14:10 15:37 （以下略）
3 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:327 12:1 13:3 14:6 15:337 （以下略）
1 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:13 15:763 （以下略）
1 qid:1 1:3 2:0 3:0 4:0 5:3 6:1 7:0 8:0 9:0 10:1 11:2032 12:0 13:6 14:13 15:2051 （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ head -10 data/vali.txt
1 qid:10 1:2 2:0 3:0 4:0 5:2 6:0.666667 7:0 8:0 9:0 10:0.666667 11:835 12:0 13:8 14:10 15:853  （以下略）
1 qid:10 1:1 2:0 3:1 4:3 5:3 6:0.333333 7:0 8:0.333333 9:1 10:1 11:10 12:0 13:9 14:11 15:30  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:557 12:0 13:7 14:11 15:575  （以下略）
1 qid:10 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:522 12:0 13:6 14:8 15:536  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:59 12:0 13:5 14:5 15:69  （以下略）
3 qid:10 1:3 2:0 3:3 4:1 5:3 6:1 7:0 8:1 9:0.333333 10:1 11:203 12:0 13:7 14:5 15:215  （以下略）
2 qid:10 1:3 2:0 3:3 4:1 5:3 6:1 7:0 8:1 9:0.333333 10:1 11:321 12:0 13:5 14:5 15:331  （以下略）
2 qid:10 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:195 12:0 13:6 14:4 15:205  （以下略）
1 qid:10 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:10 12:0 13:2 14:7 15:19  （以下略）
1 qid:10 1:2 2:0 3:1 4:0 5:2 6:0.666667 7:0 8:0.333333 9:0 10:0.666667 11:919 12:0 13:6 14:4 15:929  （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ cat data/test.txt
1 qid:1 1:3 2:3 3:0 4:0 5:3 6:1 7:1 8:0 9:0 10:1 11:156 12:4 13:0 14:7 15:167 （以下略）
2 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:406 12:0 13:5 14:5 15:416 （以下略）
3 qid:1 1:3 2:0 3:2 4:0 5:3 6:1 7:0 8:0.666667 9:0 10:1 11:146 12:0 13:3 14:7 15:156 （以下略）
4 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:287 12:1 13:4 14:7 15:299  （以下略）
5 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:2009 12:2 13:4 14:7 15:2022  （以下略）
6 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:935 12:3 13:4 14:7 15:949  （以下略）
7 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1363 12:4 13:4 14:7 15:1378 （以下略）
8 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:489 12:0 13:4 14:10 15:503  （以下略）
9 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1295 12:2 13:4 14:7 15:1308 （以下略）
10 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:510 12:0 13:4 14:5 15:519  （以下略）
11 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:164 12:1 13:10 14:10 15:185  （以下略）
12 qid:1 1:3 2:1 3:3 4:0 5:3 6:1 7:0.333333 8:1 9:0 10:1 11:666 12:3 13:4 14:7 15:680  （以下略）
13 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1475 12:1 13:4 14:7 15:1487  （以下略）
14 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1601 12:0 13:1593 14:5 15:3199  （以下略）
15 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:1268 12:0 13:4 14:7 15:1279  （以下略）
16 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:12 15:762  （以下略）
17 qid:1 1:0 2:0 3:0 4:0 5:0 6:0 7:0 8:0 9:0 10:0 11:21 12:3 13:3 14:10 15:37  （以下略）
18 qid:1 1:3 2:0 3:3 4:0 5:3 6:1 7:0 8:1 9:0 10:1 11:327 12:1 13:3 14:6 15:337  （以下略）
19 qid:1 1:3 2:0 3:2 4:2 5:3 6:1 7:0 8:0.666667 9:0.666667 10:1 11:741 12:0 13:9 14:13 15:763  （以下略）
20 qid:1 1:3 2:0 3:0 4:0 5:3 6:1 7:0 8:0 9:0 10:1 11:2032 12:0 13:6 14:13 15:2051  （以下略）
MacBookPro-makmorit-jp:pyltr makmorit$ ```

## (3) example を使用した実行例

以下のサイトの記述を参考にしています。

https://github.com/jma127/pyltr/blob/master/README.rst#example

### (3-0) データの読み込み

データファイルをベクトル変換するまでの流れは、pyltrを利用します。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../../../pyltr") #<--- ~/GitHub/pyltr
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
print(learning_dir)

/Users/makmorit/GitHub/pyltr


In [17]:
import pyltr
import numpy as np

with open('data/train.txt') as trainfile:
    X, y, query_ids, _ = pyltr.data.letor.read_dataset(trainfile)
print('Train data: samples=%d, features=%d, query_ids=%d' % (
    X.shape[0], 
    X.shape[1], 
    np.size(np.unique(query_ids))
))

Train data: samples=10000, features=136, query_ids=87


### (3-1) 学習処理

学習セットファイル data/train.txt を引数として学習処理を行うと、戻り値にfeatureごとの影響変数が格納されます。

（サポートベクトルか？　ドキュメントがないので、詳細は不明です）

In [18]:
from pysofia import sofia_ml

In [20]:
alpha = 0.01
n_samples = X.shape[0]
n_features = X.shape[1]
learner = sofia_ml.learner_type.pegasos
loop = sofia_ml.loop_type.rank
eta = sofia_ml.eta_type.basic_eta

coef = sofia_ml.svm_train(X, y, query_ids, alpha, n_samples, n_features, learner, loop, eta,
              max_iter=100, step_probability=0.5)

In [22]:
np.size(coef)

136

In [24]:
coef[0:10] # 先頭１０件をサンプル表示

array([  1.73490072e-04,   3.95229821e-10,   1.73790890e-04,
         3.01610356e-07,   1.73490072e-04,  -2.02447378e-10,
         1.97614911e-10,   3.00815827e-07,   3.01412769e-07,
        -2.02447378e-10])

### (3-2) 予測処理

テストデータファイル 'data/test.txt' を引数として予測処理処理を行うと、戻り値に予測結果が格納されます。

ちなみに、例をわかりやすくする為、テストデータの１番目の列（下記y_testに対応）にはユニークなラベルを付しています。

（y_test が svm_predict 関数の引数として不要なことから、予測処理には無関係です）

In [25]:
with open('data/test.txt') as evalfile:
    X_test, y_test, query_ids_test, _ = pyltr.data.letor.read_dataset(evalfile)

In [26]:
predict_type = sofia_ml.predict_type.linear

pred = sofia_ml.svm_predict(X_test, coef, predict_type=predict_type)
pred

array([-582182.        , -582222.4375    ,  -33482.54296875,
       -581194.75      , -497012.5625    , -499096.125     ,
       -500175.25      , -632853.25      , -575648.4375    ,
       -633458.1875    , -622479.3125    , -480063.        ,
       -480182.40625   , -114995.796875  , -480147.625     ,
       -658627.8125    , -609913.625     ,   -2074.81298828,
       -656253.5       , -203220.515625  ])

### (3-3) 予測結果を参照

pred に、ランキング・スコアが出力されます。

（ちなみに、テストデータのレコードと同じ並びになっているようです）

これをユーザープログラムなどで降順に整列し、ランキング結果として利用する想定のようです。

In [27]:
'''
    ランキングスコアで降順ソート
'''
ranking_array = []
for index, _ in enumerate(X_test):
    ranking_array.append((query_ids_test[index], y_test[index], pred[index]))

sorted_array = sorted(ranking_array, key=lambda x:x[2], reverse=True)

In [28]:
'''
    ランキングを表示します
'''
for qid, label, score in sorted_array:
    print('qid=%s label=%d ranking score=%0.3f' % (qid, label, score))

qid=1 label=18 ranking score=-2074.813
qid=1 label=3 ranking score=-33482.543
qid=1 label=14 ranking score=-114995.797
qid=1 label=20 ranking score=-203220.516
qid=1 label=12 ranking score=-480063.000
qid=1 label=15 ranking score=-480147.625
qid=1 label=13 ranking score=-480182.406
qid=1 label=5 ranking score=-497012.562
qid=1 label=6 ranking score=-499096.125
qid=1 label=7 ranking score=-500175.250
qid=1 label=9 ranking score=-575648.438
qid=1 label=4 ranking score=-581194.750
qid=1 label=1 ranking score=-582182.000
qid=1 label=2 ranking score=-582222.438
qid=1 label=17 ranking score=-609913.625
qid=1 label=11 ranking score=-622479.312
qid=1 label=8 ranking score=-632853.250
qid=1 label=10 ranking score=-633458.188
qid=1 label=19 ranking score=-656253.500
qid=1 label=16 ranking score=-658627.812


#### 参考：pyltrでの結果

```
qid=1 label=18 ranking score=1.658 <---ここだけが一致していて、あとは全く整合性のない結果となった
qid=1 label=1 ranking score=-0.474
qid=1 label=10 ranking score=-0.495
qid=1 label=8 ranking score=-0.508
qid=1 label=2 ranking score=-0.620
qid=1 label=4 ranking score=-0.645
qid=1 label=9 ranking score=-0.665
qid=1 label=12 ranking score=-0.690
qid=1 label=6 ranking score=-0.692
qid=1 label=7 ranking score=-0.692
qid=1 label=13 ranking score=-0.692
qid=1 label=15 ranking score=-0.692
qid=1 label=5 ranking score=-0.700
qid=1 label=11 ranking score=-0.756
qid=1 label=14 ranking score=-1.010
qid=1 label=3 ranking score=-1.164
qid=1 label=16 ranking score=-1.345
qid=1 label=19 ranking score=-1.345
qid=1 label=17 ranking score=-1.452
qid=1 label=20 ranking score=-1.486
```